In [ ]:
import os
import pandas as pd

from utils import (
    get_experiments_metadata,
    check_duplicate_runs,
    plot_experiment,
    get_experiment_results,
)

BASE_OUTPUT_DIR = "/opt/gpudata/labrag/"

In [ ]:
check_duplicate_runs(BASE_OUTPUT_DIR)

In [ ]:
for dataset in [
    "mimic-cxr",
    "chexpertplus",
]:
    for section in [
        "findings",
        "impression",
    ]:
        dataset_section_results = []
        fig_dir = f"../figs/{dataset}-{section}"
        os.makedirs(fig_dir, exist_ok=True)
        experiments = get_experiments_metadata(
            dataset=dataset,
            section=section,
            result_dir=BASE_OUTPUT_DIR,
        )
        for exp_name, (exp_dir, exp_trials) in experiments.items():
            save_name = exp_name.replace(' ', '-').lower()
            trial_dfs = get_experiment_results(
                exp_dir=exp_dir,
                exp_trials=exp_trials,
                # normalize_bertscore_lang="en",
            )
            if dataset == "mimic-cxr":
                title = f"MIMIC-CXR - {section.title()}"
            elif dataset == "chexpertplus":
                title = f"CheXpert Plus - {section.title()}"
            else:
                raise ValueError(f"Please add custom title for dataset {dataset}")

            # Clinical metrics
            fig = plot_experiment(
                title=title,
                exp_name=exp_name,
                exp_trials=exp_trials,
                trial_dfs=trial_dfs,
                metrics=["f1radgraph", "f1chexbert"],
            )
            fig.savefig(os.path.join(fig_dir, f"{save_name}.pdf"))
            fig.savefig(os.path.join(fig_dir, f"{save_name}.png"), dpi=300)

            # Full metrics
            fig = plot_experiment(
                title=title,
                exp_name=exp_name,
                exp_trials=exp_trials,
                trial_dfs=trial_dfs,
                metrics=["bleu4", "rougeL", "bertscore", "f1radgraph", "f1chexbert"],
            )
            fig.savefig(os.path.join(fig_dir, f"{save_name}-full.pdf"))
            fig.savefig(os.path.join(fig_dir, f"{save_name}-full.png"), dpi=300)

            # Tabular results
            df = pd.DataFrame([df[["bleu4", "rougeL", "bertscore", "f1radgraph", "f1chexbert"]].mean() for df in trial_dfs])
            df["experiment"] = exp_name
            df["variable"] = [v for v, f in exp_trials]
            df = df[["experiment", "variable", "bleu4", "rougeL", "bertscore", "f1radgraph", "f1chexbert"]]
            dataset_section_results.append(df)

        # write results, use other notebook for formatting TeX tables
        dataset_section_results = pd.concat(dataset_section_results, ignore_index=True)
        dataset_section_results.to_csv(os.path.join(fig_dir, f"all-results.csv"), index=False)